In [1]:
import pandas as pd

url = 'https://drive.google.com/file/d/1FrFTfUln67599LTm2uMTSqM8DjqpAaKL/view'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
data = pd.read_csv(path)
import pandas as pd

url = 'https://drive.google.com/file/d/1FrFTfUln67599LTm2uMTSqM8DjqpAaKL/view'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
data = pd.read_csv(path)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23524 entries, 0 to 23523
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   country                 23524 non-null  object
 1   year                    23524 non-null  int64 
 2   uniqueid                23524 non-null  object
 3   bank_account            23524 non-null  object
 4   location_type           23524 non-null  object
 5   cellphone_access        23524 non-null  object
 6   household_size          23524 non-null  int64 
 7   age_of_respondent       23524 non-null  int64 
 8   gender_of_respondent    23524 non-null  object
 9   relationship_with_head  23524 non-null  object
 10  marital_status          23524 non-null  object
 11  education_level         23524 non-null  object
 12  job_type                23524 non-null  object
dtypes: int64(3), object(10)
memory usage: 2.3+ MB


In [4]:
data.head()

,country,year,uniqueid,bank_account,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Kenya,2018,uniqueid_1,Yes,Rural,Yes,3,24,Female,Spouse,Married/Living together,Secondary education,Self employed
1,Kenya,2018,uniqueid_2,No,Rural,No,5,70,Female,Head of Household,Widowed,No formal education,Government Dependent
2,Kenya,2018,uniqueid_3,Yes,Urban,Yes,5,26,Male,Other relative,Single/Never Married,Vocational/Specialised training,Self employed
3,Kenya,2018,uniqueid_4,No,Rural,Yes,5,34,Female,Head of Household,Married/Living together,Primary education,Formally employed Private
4,Kenya,2018,uniqueid_5,No,Urban,No,8,26,Male,Child,Single/Never Married,Primary education,Informally employed


In [5]:
%pip install ydata_profiling
import ydata_profiling

profile = ydata_profiling.ProfileReport(data)
profile.to_file(output_file="financialinclusiondata.html")


Note: you may need to restart the kernel to use updated packages.


c:\Users\Bunmi\Documents\Financial Inclusion\constellation\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Summarize dataset:  75%|███████▌  | 15/20 [00:02<00:00, 11.20it/s, Calculate auto correlation]             c:\Users\Bunmi\Documents\Financial Inclusion\constellation\Lib\site-packages\ydata_profiling\model\correlations.py:66: UserWarning: There was an attempt to calculate the auto correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"auto": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/ydata-profiling/issues
(include the error message: 'could not convert string to float: 'Yes'')
  warnings.warn(
Export report to file: 100%|██████████| 1/1 [00:00<00:00, 28.85it/s]


In [6]:
X = data.drop(columns=['uniqueid','bank_account', 'country','year'], axis=1)
y = data.bank_account

In [7]:
from sklearn.preprocessing import LabelEncoder

object_columns = ["location_type", "cellphone_access", "gender_of_respondent", 
                  "relationship_with_head", 
                  "marital_status", "education_level", "job_type"]

encoder = LabelEncoder()
for i in object_columns:
    X[i]= encoder.fit_transform(X[i])

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

# Create the individual classifiers
clf1 = RandomForestClassifier(n_estimators=100)
clf2 = SVC(probability=True)

# Create the voting classifier
eclf = VotingClassifier(estimators=[('rf', clf1), ('svc', clf2)], voting='hard')

# Fit the voting classifier on the data
eclf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = eclf.predict(X_test)

In [11]:
X.columns

Index(['location_type', 'cellphone_access', 'household_size',
       'age_of_respondent', 'gender_of_respondent', 'relationship_with_head',
       'marital_status', 'education_level', 'job_type'],
      dtype='object')

In [13]:
import pickle 
with open('model.pkl', 'wb') as file:  
    pickle.dump(eclf, file)